# OMOP Data Exploration and Analysis with PostgreSQL

This jupyter notebook includes a example for querying and analyzing OMOP(Observational Medical Outcomes Partnership) data stored in a PostgreSQL database. The notebook covers essential steps, from establishing a connection to the database to executing SQL queries, and visualizing key insights.

https://www.ohdsi.org/data-standardization/

### Importing required Libraries

- **psycopg2** For establishing a connection between python and a PostgreSQL database.
- **pandas** For efficient data handling, manipulation nad analysis of data.
- **sqlio** For executing SQL queries and reading the results directly into pandas dataframe.

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps
import warnings

warnings.filterwarnings('ignore') #ignoring warning related to sqlio and sql alchemy. Will improve this.

### Database connection and setup

Please change the username password to your OMOP DB.

In [2]:
conn = ps.connect(dbname="ohdsi",
                   user = "ohdsi_admin_user",
                   password = "admin1",
                   host = "omop-db-postgress",
                   port = "5432")

## Initial Analysis - Query Counts from OMOP DB
### Number of Person

In [3]:
df = sqlio.read_sql_query("SELECT COUNT(*) FROM cds_cdm.person", conn)
df.head()

,count
0,130


### Number of Observation

In [4]:
df = sqlio.read_sql_query("SELECT COUNT(*) FROM cds_cdm.observation", conn)
df.head()

,count
0,9040


### Number of Death

In [5]:
df = sqlio.read_sql_query("SELECT COUNT(*) FROM cds_cdm.death", conn)
df.head()

,count
0,30


### Number of Procedure

In [6]:
df = sqlio.read_sql_query("SELECT COUNT(*) FROM cds_cdm.procedure_occurrence", conn)
df.head()

,count
0,9400


### Number of measurements eg: Height, Weight etc.

In [7]:
df = sqlio.read_sql_query("SELECT COUNT(*) FROM cds_cdm.measurement", conn)
df.head()

,count
0,97087


### Describe Height meaurements

In [8]:
df = sqlio.read_sql_query("SELECT * FROM cds_cdm.measurement where measurement_source_concept_id=3036277", conn)
df["value_as_number"].describe()

count    2048.000000
mean      160.541455
std        27.589257
min        51.100000
25%       162.700000
50%       170.000000
75%       175.400000
max       189.300000
Name: value_as_number, dtype: float64

## Encounter visit longitudinal Analysis

In [9]:
df = sqlio.read_sql_query(f"SELECT * FROM cds_cdm.visit_occurrence", conn)
df.head()

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitting_source_concept_id,admitting_source_value,discharge_to_concept_id,discharge_to_source_value,preceding_visit_occurrence_id,fhir_identifier,fhir_logical_id
0,1,1,9202,2014-02-28,2014-02-28 19:12:18,2014-02-28,2014-02-28 19:27:18,32817,None,None,None,None,None,None,None,None,None,None,enc-148814
1,2,1,9202,2016-05-27,2016-05-27 19:12:18,2016-05-27,2016-05-27 19:27:18,32817,None,None,None,None,None,None,None,None,None,None,enc-148902
2,3,1,9202,2019-12-13,2019-12-13 19:12:18,2019-12-13,2019-12-13 19:27:18,32817,None,None,None,None,None,None,None,None,None,None,enc-148983
3,4,1,9202,2020-07-17,2020-07-17 12:12:18,2020-07-17,2020-07-17 12:27:18,32817,None,None,None,None,None,None,None,None,None,None,enc-148998
4,5,1,9203,2020-12-10,2020-12-10 19:12:18,2020-12-10,2020-12-10 20:12:18,32817,None,None,None,None,None,None,None,None,None,None,enc-149006


In [10]:
df = sqlio.read_sql_query(f"SELECT person_id, COUNT(person_id) AS count FROM cds_cdm.visit_occurrence GROUP BY person_id", conn)
df["count"].describe()

count    130.000000
mean      83.230769
std      146.124060
min        1.000000
25%       20.250000
50%       35.000000
75%       58.000000
max      851.000000
Name: count, dtype: float64

### What has the person with max number of visits come for?

In [11]:
max_count = df["count"].describe().max() 
person_id_max_visit = df.loc[df['count'] == max_count]["person_id"].item()
print(person_id_max_visit)
df_new = sqlio.read_sql_query(f"SELECT * FROM cds_cdm.procedure_occurrence where person_id={person_id_max_visit}", conn)
procedure_concept_id = df_new['procedure_concept_id'].iat[0]
procedure_reason = sqlio.read_sql_query(f"SELECT * FROM cds_cdm.concept where concept_id={procedure_concept_id}", conn)
print(procedure_reason)

43
   concept_id    concept_name  domain_id vocabulary_id concept_class_id  \
0     4146536  Renal dialysis  Procedure        SNOMED        Procedure   

  standard_concept concept_code valid_start_date valid_end_date invalid_reason  
0                S    265764009       2002-01-31     2099-12-31           None  


## Procedure specific longitudinal Analysis

### Enter OMOP code for a procedure eg: Mammography for breast cancer
#### Search example
https://athena.ohdsi.org/search-terms/terms?conceptClass=Procedure&page=1&pageSize=15&query=Mammography&boosts 

In [12]:
omop_code = 4324693 #Code for Mammography. Please change this to the code that you are interested in.
df = sqlio.read_sql_query(f"SELECT * FROM cds_cdm.procedure_occurrence where procedure_concept_id={omop_code}", conn)
df.head()

,procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_datetime,procedure_type_concept_id,modifier_concept_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,procedure_source_value,procedure_source_concept_id,modifier_source_value,fhir_identifier,fhir_logical_id
0,750,10,4324693,2020-02-25,2020-02-25 19:45:04,32817,None,None,None,2733,None,71651007,4324693,None,None,pro-171665
1,764,10,4324693,2021-02-08,2021-02-08 22:24:27,32817,None,None,None,2741,None,71651007,4324693,None,None,pro-171732
2,793,10,4324693,2023-01-24,2023-01-24 07:44:57,32817,None,None,None,2764,None,71651007,4324693,None,None,pro-171987
3,853,10,4324693,2022-02-20,2022-02-20 07:55:10,32817,None,None,None,2757,None,71651007,4324693,None,None,pro-171917
4,2010,23,4324693,2016-04-13,2016-04-13 21:39:15,32817,None,None,None,4518,None,71651007,4324693,None,None,pro-193832


In [13]:
df = sqlio.read_sql_query(f"SELECT person_id, COUNT(person_id) AS count FROM cds_cdm.procedure_occurrence where procedure_concept_id={omop_code} GROUP BY person_id", conn)
df

,person_id,count
0,10,4
1,23,10
2,88,10
